In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [24]:
df = pd.read_csv('data/single_step_df_ints_2022-02-22.csv')

In [25]:
df

,input,target,start_coords_x,start_coords_y,start_coords_z,coordinates_p.x,coordinates_p.y,coordinates_p.z,p.containment,p.food_k,...,h.food_k,h.strong_k,h.mid_k,coordinates_q.x,coordinates_q.y,coordinates_q.z,q.containment,q.food_k,q.strong_k,q.mid_k
0,<start>,p,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,h,h,0.000,2.000,2.00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,h,s,0.000,2.000,2.00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,s,d,0.000,2.000,2.00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,d,g,0.000,2.000,2.00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [35]:
df['target'].unique()
df['target'].value_counts()

df['target'].loc[df['target'] == 'l']

727    l
Name: target, dtype: object

In [28]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [48]:
# remove row with target that only occurs once

df_new = df.drop(index=727, axis=0)
df_new.reset_index(inplace=True, drop=True)

df_new

,input,target,start_coords_x,start_coords_y,start_coords_z,coordinates_p.x,coordinates_p.y,coordinates_p.z,p.containment,p.food_k,...,h.food_k,h.strong_k,h.mid_k,coordinates_q.x,coordinates_q.y,coordinates_q.z,q.containment,q.food_k,q.strong_k,q.mid_k
0,<start>,p,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1,p,o,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2,o,c,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,c,g,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,g,k,0.513,-0.531,0.74,0.008034,0.957082,0.689054,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,h,h,0.000,2.000,2.00,-99.000000,-99.000000,-99.000000,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1457,h,s,0.000,2.000,2.00,-99.000000,-99.000000,-99.000000,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1458,s,d,0.000,2.000,2.00,-99.000000,-99.000000,-99.000000,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1459,d,g,0.000,2.000,2.00,-99.000000,-99.000000,-99.000000,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [49]:
# generate stratified split for train - test/val

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new, df_new['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [50]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [70]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [71]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [72]:
batch_size = 256

In [73]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_75777/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}


In [74]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of targets:', label_batch)

Every feature: ['input', 'start_coords_x', 'start_coords_y', 'start_coords_z', 'coordinates_p.x', 'coordinates_p.y', 'coordinates_p.z', 'p.containment', 'p.food_k', 'p.strong_k', 'p.mid_k', 'coordinates_o.x', 'coordinates_o.y', 'coordinates_o.z', 'o.containment', 'o.food_k', 'o.strong_k', 'o.mid_k', 'coordinates_c.x', 'coordinates_c.y', 'coordinates_c.z', 'c.containment', 'c.food_k', 'c.strong_k', 'c.mid_k', 'coordinates_g.x', 'coordinates_g.y', 'coordinates_g.z', 'g.containment', 'g.food_k', 'g.strong_k', 'g.mid_k', 'coordinates_k.x', 'coordinates_k.y', 'coordinates_k.z', 'k.containment', 'k.food_k', 'k.strong_k', 'k.mid_k', 'coordinates_r.x', 'coordinates_r.y', 'coordinates_r.z', 'r.containment', 'r.food_k', 'r.strong_k', 'r.mid_k', 'coordinates_w.x', 'coordinates_w.y', 'coordinates_w.z', 'w.containment', 'w.food_k', 'w.strong_k', 'w.mid_k', 'coordinates_f.x', 'coordinates_f.y', 'coordinates_f.z', 'f.containment', 'f.food_k', 'f.strong_k', 'f.mid_k', 'coordinates_s.x', 'coordinates_s

In [75]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [76]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [77]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [78]:
all_inputs, encoded_features = create_input_data(df)

In [79]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(20)(x)

model = tf.keras.Model(all_inputs, output)

In [80]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [82]:
model.fit(train_ds, epochs=100, validation_data=val_ds)

Epoch 1/100
4/4 [==============================] - 6s 797ms/step - loss: 3.0356 - accuracy: 0.1027 - val_loss: 2.5187 - val_accuracy: 0.1773
Epoch 2/100
4/4 [==============================] - 0s 37ms/step - loss: 2.4666 - accuracy: 0.1614 - val_loss: 2.3031 - val_accuracy: 0.1636
Epoch 3/100
4/4 [==============================] - 0s 39ms/step - loss: 2.2997 - accuracy: 0.1703 - val_loss: 2.1996 - val_accuracy: 0.1909
Epoch 4/100
4/4 [==============================] - 0s 38ms/step - loss: 2.2095 - accuracy: 0.1703 - val_loss: 2.1382 - val_accuracy: 0.1909
Epoch 5/100
4/4 [==============================] - 0s 36ms/step - loss: 2.1699 - accuracy: 0.1742 - val_loss: 2.1190 - val_accuracy: 0.1682
Epoch 6/100
4/4 [==============================] - 0s 36ms/step - loss: 2.1214 - accuracy: 0.1742 - val_loss: 2.0988 - val_accuracy: 0.1636
Epoch 7/100
4/4 [==============================] - 0s 37ms/step - loss: 2.0988 - accuracy: 0.1888 - val_loss: 2.0826 - val_accuracy: 0.2045
Epoch 8/100
4/4 [==

4/4 [==============================] - 0s 39ms/step - loss: 1.5076 - accuracy: 0.4462 - val_loss: 1.8444 - val_accuracy: 0.3591
Epoch 60/100
4/4 [==============================] - 0s 41ms/step - loss: 1.4748 - accuracy: 0.4599 - val_loss: 1.8411 - val_accuracy: 0.3545
Epoch 61/100
4/4 [==============================] - 0s 39ms/step - loss: 1.4808 - accuracy: 0.4511 - val_loss: 1.8396 - val_accuracy: 0.3682
Epoch 62/100
4/4 [==============================] - 0s 39ms/step - loss: 1.4763 - accuracy: 0.4423 - val_loss: 1.8430 - val_accuracy: 0.3727
Epoch 63/100
4/4 [==============================] - 0s 43ms/step - loss: 1.4730 - accuracy: 0.4501 - val_loss: 1.8456 - val_accuracy: 0.3636
Epoch 64/100
4/4 [==============================] - 0s 39ms/step - loss: 1.4617 - accuracy: 0.4530 - val_loss: 1.8363 - val_accuracy: 0.3409
Epoch 65/100
4/4 [==============================] - 0s 37ms/step - loss: 1.4619 - accuracy: 0.4726 - val_loss: 1.8356 - val_accuracy: 0.3364
Epoch 66/100
4/4 [========

In [84]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 83ms/step - loss: 1.9686 - accuracy: 0.3151
Accuracy 0.31506848335266113


In [85]:
model.save('next_obj_classifier')

2022-02-22 14:12:48.148835: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: next_obj_classifier/assets


INFO:tensorflow:Assets written to: next_obj_classifier/assets


In [86]:
reloaded_model = tf.keras.models.load_model('next_obj_classifier')

In [38]:
# test prediction for one sample

In [87]:
sample = df.loc[0].drop('target').to_dict()

In [88]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [89]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [90]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [91]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(0, len(dataframe)): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [92]:
errors = get_prequential_error(df, model, labels, nr_of_sequences)

In [93]:
summed_error = [sum(error) for error in errors[:-1]]

In [94]:
np.median(summed_error)

4.0

In [82]:
#with open('results/nn_spatialinfo_prequential_summed.txt', 'w') as file:
#    file.write(str(summed_error))